# Sort ( based on content length ), Offset and length

Advance dataset operations, of sorting, offset, and length support

In [1]:
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="infctx-v5-sort-offset-test"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/ubuntu/picocreator-memory-experiment/notebook/trainer-v5-validation
TRAINER_DIR: /home/ubuntu/picocreator-memory-experiment/RWKV-v5
PROJECT_DIR: /home/ubuntu/picocreator-memory-experiment


In [2]:
# Init the model
!cd "{TRAINER_DIR}" && \
    python3 ./init_model.py \
        --n_layer 6 --n_embd 2048 \
        --vocab_size neox --skip-if-exists \
        "../model/L6-D2048-neox-v5base-init.pth"

[2023-09-14 07:12:02,378] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
---- Initializing model ----
No of layers: 6
Embedding size: 2048
Output model path: ../model/L6-D2048-neox-v5base-init.pth
Vocab size: 50277
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Model exists, skipping init_model


In [3]:
# Lets preload the requried dataset 
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/test-sort-offset-length.yaml"

Saving the dataset (1/1 shards): 100%|█| 989/989 [00:00<00:00, 56411.55 examples
Saving the dataset (1/1 shards): 100%|█| 100/100 [00:00<00:00, 10393.78 examples


In [4]:
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/test-sort-offset-length.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (deepspeed_stage_2_offload)" \
        --trainer.strategy="deepspeed_stage_2_offload" \
        --trainer.devices="{GPU_DEVICES}"

[2023-09-14 07:12:15,982] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
/home/ubuntu/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/ubuntu/picocreator-memory-experiment/notebook/trainer-v5-validation/config/test-sort-offset-length.yaml', '--trainer.logger.init_args.name=infctx-v5-sort-offset-test (deepspeed_stage_2_offload)', '--trainer.strategy=deepspeed_stage_2_offload', '--trainer.devices=auto'], args=['fit', '-c', '/home/ubuntu/picocreator-memory-experiment/notebook/trainer-v5-validation/config/test-sort-offset-length.yaml', '--trainer.logger.init_args.name=infctx-v5-sort-offset-test